In [2]:
import numpy as np
means = [[2, 2], [8, 3], [3, 6]]
cov = [[1, 0], [0, 1]]
N = 500
X0 = np.random.multivariate_normal(means[0], cov, N)
X1 = np.random.multivariate_normal(means[1], cov, N)
X2 = np.random.multivariate_normal(means[2], cov, N)

# each column is a datapoint
X = np.concatenate((X0, X1, X2), axis = 0).T
# extended data
X = np.concatenate((np.ones((1, 3*N)), X), axis = 0)
C = 3

original_label = np.asarray([0]*N + [1]*N + [2]*N).T

In [3]:
def softmax(Z):
    e_Z = np.exp(Z)
    A = e_Z / e_Z.sum(axis = 0)
    return A

In [28]:
## One-hot coding
y = original_label
from scipy import sparse
def convert_labels(y, C = C):
    Y = sparse.coo_matrix((np.ones_like(y),
        (y, np.arange(len(y)))), shape = (C, len(y))).toarray()
    return Y

Y = convert_labels(y, C)
print(Y)

[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]


In [31]:
W_init = np.random.randn(X.shape[0], C)
# cost or loss function
def cost(X, Y, W):
    A = softmax(W.T.dot(X))
    return -np.sum(Y*np.log(A))

In [36]:
def grad(X, Y, W):
    A = softmax((W.T.dot(X)))
    E = A - Y
    return X.dot(E.T)

In [37]:
def numerical_grad(X, Y, W, cost):
    eps = 1e-6
    g = np.zeros_like(W)
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            W_p = W.copy()
            W_n = W.copy()
            W_p[i, j] += eps
            W_n[i, j] -= eps
            g[i,j] = (cost(X, Y, W_p) - cost(X, Y, W_n))/(2*eps)
    return g

g1 = grad(X, Y, W_init)
g2 = numerical_grad(X, Y, W_init, cost)

print(np.linalg.norm(g1 - g2))

2.251816958961235e-06


In [70]:
def softmax_regression(X, y, W_init, eta, tol = 1e-4, max_count = 10000):
    W = [W_init]
    C = W_init.shape[1]
    Y = convert_labels(y, C)
    N = X.shape[1]
    d = X.shape[0]

    count = 0
    check_w_after = 20
    while count < max_count:
        # mix data
        mix_id = np.random.permutation(1000)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = Y[:, i].reshape(C, 1)
            ai = softmax(np.dot(W[-1].T, xi))
            W_new = W[-1] + eta*xi.dot((yi - ai).T)
            count += 1
            # stopping criteria
            if count%check_w_after == 0:
                if np.linalg.norm(W_new - W[-check_w_after]) < tol:
                    return W
            W.append(W_new)
    return W
eta = .05
d = X.shape[0]
W_init = np.random.randn(d, C)

W = softmax_regression(X, y, W_init, eta)
# W[-1] is the solution, W is all history of weights

In [47]:
def softmax_stable(Z):
    e_Z = np.exp(Z - np.max(Z, axis = 0, keepdims = True))
    A = e_Z / e_Z.sum(axis = 0)
    return A

In [50]:
print(W[-1])

[[ 7.81560096 -4.4212588  -4.01323539]
 [-0.66010967  1.88744747 -0.6476996 ]
 [-1.67070119 -1.52805836  1.29750006]]


In [53]:
def pred(W, X):
    A = softmax_stable(W.T.dot(X))
    return np.argmax(A, axis = 0)
pred(W[-1],X)

[0 0 0 ... 2 2 2]


In [54]:
import zipfile

# Unzip the archive
local_zip = 'C:\\Users\\TGDD\\Desktop\\học máy\\cuối kỳ\\./animals.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()

zip_ref.close()

In [55]:
import os

base_dir = 'animals'

print("Contents of base directory:")
print(os.listdir(base_dir))

print("\nContents of train directory:")
print(os.listdir(f'{base_dir}/train'))

print("\nContents of validation directory:")
print(os.listdir(f'{base_dir}/validation'))

Contents of base directory:
['train', 'validation']

Contents of train directory:
['cats', 'chickens', 'dogs']

Contents of validation directory:
['cats', 'chickens', 'dogs']


In [56]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with training pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
train_chickens_dir = os.path.join(train_dir, 'chickens')

# Directory with validation pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
validation_chickens_dir = os.path.join(validation_dir, 'chickens')

In [57]:
train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)
train_chicken_fnames = os.listdir(train_chickens_dir)

validation_cat_fnames = os.listdir(validation_cats_dir)
validation_dog_fnames = os.listdir(validation_dogs_dir)
validation_chicken_fnames = os.listdir(validation_chickens_dir)

print(train_cat_fnames[:10])
print(train_dog_fnames[:10])
print(train_chicken_fnames[:10])

print('total training cat images :', len(train_cat_fnames))
print('total training dog images :', len(train_dog_fnames))
print('total training chicken images :', len(train_chicken_fnames))

print('total validation cat images :', len(validation_cat_fnames))
print('total validation dog images :', len(validation_dog_fnames))
print('total validation chicken images :', len(validation_chicken_fnames))

['cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.101.jpg', 'cat.102.jpg', 'cat.103.jpg', 'cat.104.jpg', 'cat.105.jpg', 'cat.106.jpg']
['dog.0.jpg', 'dog.1.jpg', 'dog.10.jpg', 'dog.100.jpg', 'dog.101.jpg', 'dog.102.jpg', 'dog.103.jpg', 'dog.104.jpg', 'dog.105.jpg', 'dog.106.jpg']
['10.jpeg', '100.jpeg', '1000.jpeg', '1001.jpeg', '1010.jpeg', '1013.jpeg', '1016.jpeg', '1017.jpeg', '1018.jpeg', '102.jpeg']
total training cat images : 1000
total training dog images : 1000
total training chicken images : 1000
total validation cat images : 500
total validation dog images : 500
total validation chicken images : 500


In [61]:
image_size = 512

In [67]:
import cv2
from tqdm import tqdm
from skimage.feature import hog

X_train = np.zeros((3000, int(image_size**2 / 32)))

train_cat_img_paths = [
    os.path.join(train_cats_dir, fname) for fname in train_cat_fnames
    ]
train_dog_img_paths = [
    os.path.join(train_dogs_dir, fname) for fname in train_dog_fnames
    ]
train_chicken_img_paths = [
    os.path.join(train_chickens_dir, fname) for fname in train_chicken_fnames
    ]

count = 0

for img_path in tqdm(train_cat_img_paths + train_dog_img_paths + train_chicken_img_paths):
    img = cv2.resize(cv2.imread(img_path), (image_size, image_size))

    fd = hog(img, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), channel_axis=-1)

    X_train[count] = fd
    count += 1

y_train = 1000 * [0] + 1000 * [1] + 1000 * [2]

100%|██████████| 3000/3000 [04:26<00:00, 11.26it/s]


In [63]:
from tqdm import tqdm

X_test = np.zeros((1500, int(image_size**2 / 32)))

validation_cat_img_paths = [
    os.path.join(validation_cats_dir, fname) for fname in validation_cat_fnames
    ]
validation_dog_img_paths = [
    os.path.join(validation_dogs_dir, fname) for fname in validation_dog_fnames
    ]
validation_chicken_img_paths = [
    os.path.join(validation_chickens_dir, fname) for fname in validation_chicken_fnames
    ]

count = 0

for img_path in tqdm(validation_cat_img_paths + validation_dog_img_paths + validation_chicken_img_paths):
    img = cv2.resize(cv2.imread(img_path), (image_size, image_size))

    fd = hog(img, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), channel_axis=-1)

    X_test[count] = fd
    count += 1

y_test = 500 * ['cat'] + 500 * ['dog'] + 500 * ['chicken']

100%|██████████| 1500/1500 [02:29<00:00, 10.03it/s]


In [64]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train)

In [73]:
W_init = np.random.randn(X_train.shape[0], C)
W = softmax_regression(X_train, y_train, W_init, eta)
print(W[-1])

[[-0.25196065 -0.77417267 -0.3310273 ]
 [ 0.24591986 -1.29812637  1.2106165 ]
 [-0.43723934 -0.23176214 -0.05500634]
 ...
 [ 0.2524991  -0.37585265 -0.54593013]
 [ 0.25691144 -1.12746251 -0.38905626]
 [-0.6761117  -0.19165601  1.09340338]]


In [81]:
print(pred(W[-1],X_train))
print(pred(W[-1],X_train)[:10])
print(pred(W[-1],X_train)[1000:1010])
print(pred(W[-1],X_train)[2000:2010])

[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]


In [79]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
y_train_pred  = pred(W[-1], X_train)
# Calculate the accuracy of the model
accuracy = accuracy_score(y_train, y_train_pred[:3000])
print("Train accuracy:", accuracy)

Train accuracy: 0.33266666666666667
